In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import current_date, row_number
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder.appName("ComprehensivePySparkSQLExample").getOrCreate()

# Create sample data
data = [
    Row(name="Alice", age=25, city="New York"),
    Row(name="Bob", age=30, city="San Francisco"),
    Row(name="Charlie", age=35, city="Los Angeles"),
    Row(name="David", age=40, city="New York"),
    Row(name="Eve", age=45, city="San Francisco")
]
df = spark.createDataFrame(data)
df.createOrReplaceTempView("people")


In [ ]:

# 1. Basic Queries
result = spark.sql("SELECT * FROM people")
result.show()

result = spark.sql("SELECT name, city FROM people")
result.show()


In [ ]:

# 2. Filtering Records
result = spark.sql("SELECT * FROM people WHERE age > 30")
result.show()

result = spark.sql("SELECT * FROM people WHERE age > 30 AND city = 'San Francisco'")
result.show()


In [ ]:

# 3. Aggregation
result = spark.sql("SELECT AVG(age) as average_age FROM people")
result.show()

result = spark.sql("SELECT city, COUNT(*) as count FROM people GROUP BY city")
result.show()


In [ ]:

# 4. Sorting Results
result = spark.sql("SELECT * FROM people ORDER BY age DESC")
result.show()


In [ ]:

# 5. Joins
data2 = [
    Row(name="Alice", salary=70000),
    Row(name="Bob", salary=80000),
    Row(name="Charlie", salary=90000)
]
df2 = spark.createDataFrame(data2)
df2.createOrReplaceTempView("salaries")

result = spark.sql("""
SELECT people.name, people.age, salaries.salary
FROM people
JOIN salaries ON people.name = salaries.name
""")
result.show()

result = spark.sql("""
SELECT people.name, people.age, salaries.salary
FROM people
LEFT JOIN salaries ON people.name = salaries.name
""")
result.show()


In [ ]:

# 6. Grouping and Having
result = spark.sql("""
SELECT city, AVG(age) as average_age
FROM people
GROUP BY city
HAVING AVG(age) > 30
""")
result.show()


In [ ]:

# 7. Subqueries
result = spark.sql("""
SELECT name, age
FROM people
WHERE age > (SELECT AVG(age) FROM people)
""")
result.show()


In [ ]:

# 8. String Functions
result = spark.sql("SELECT UPPER(name) as upper_name, LENGTH(name) as name_length FROM people")
result.show()


In [ ]:

# 9. Date Functions
df = df.withColumn("current_date", current_date())
df.createOrReplaceTempView("people")

result = spark.sql("SELECT name, YEAR(current_date) as current_year FROM people")
result.show()


In [ ]:

# 10. Window Functions
window_spec = Window.partitionBy("city").orderBy("age")

df = df.withColumn("row_number", row_number().over(window_spec))
df.createOrReplaceTempView("people_with_row_number")

result = spark.sql("SELECT * FROM people_with_row_number")
result.show()


In [ ]:

# 11. Case Statements
result = spark.sql("""
SELECT name, age,
CASE
    WHEN age < 30 THEN 'Young'
    WHEN age BETWEEN 30 AND 40 THEN 'Middle-aged'
    ELSE 'Old'
END as age_group
FROM people
""")
result.show()


In [ ]:

# 12. Set Operations
data3 = [
    Row(name="Alice", age=25),
    Row(name="Bob", age=30),
    Row(name="Eve", age=45)
]
df3 = spark.createDataFrame(data3)
df3.createOrReplaceTempView("people_subset")

result = spark.sql("SELECT name, age FROM people UNION SELECT name, age FROM people_subset")
result.show()

result = spark.sql("SELECT name, age FROM people INTERSECT SELECT name, age FROM people_subset")
result.show()

result = spark.sql("SELECT name, age FROM people EXCEPT SELECT name, age FROM people_subset")
result.show()
